In [10]:
import numpy as np
import pandas as pd
import random
import unittest

class Die:
    """
    A class representing a die with a set of faces and weights.
    
    Attributes:
        faces (numpy.ndarray): The faces of the die.
        weights (numpy.ndarray): The weights associated with each face. Defaults to 1.
    """
    
    def __init__(self, faces: np.ndarray):
        """
        Initializes the Die object with a set of faces and default weights.
        
        Args:
            faces (numpy.ndarray): A NumPy array containing the faces of the die.
        
        Raises:
            TypeError: If faces is not a numpy array.
            ValueError: If the faces are not unique.
        """
        if not isinstance(faces, np.ndarray):
            raise TypeError("Faces must be a NumPy array.")
        if len(faces) != len(set(faces)):
            raise ValueError("Faces must be distinct.")
        
        self.faces = faces
        self.weights = np.ones(len(faces))
        self.df = pd.DataFrame({'faces': faces, 'weights': self.weights}).set_index('faces')
    
    def change_weight(self, face, new_weight):
        """
        Changes the weight of a specific face on the die.
        
        Args:
            face (str or int): The face whose weight is to be changed.
            new_weight (float): The new weight for the face.
        
        Raises:
            IndexError: If the face is not found.
            TypeError: If new_weight is not numeric.
        """
        if face not in self.df.index:
            raise IndexError(f"Face {face} not found in die.")
        if not isinstance(new_weight, (int, float, np.int64)):
            raise TypeError("Weight must be numeric.")
        self.df.loc[face, 'weights'] = new_weight
    
    def roll(self, times=1):
        """
        Rolls the die a specified number of times, returning the outcome.
        
        Args:
            times (int): The number of times to roll the die.
        
        Returns:
            list: A list of rolled outcomes.
        """
        return random.choices(self.df.index, weights=self.df['weights'], k=times)
    
    def show(self):
        """
        Shows the current faces and their associated weights.
        
        Returns:
            pandas.DataFrame: A DataFrame showing the faces and weights of the die.
        """
        return self.df.copy()


class Game:
    """
    A class that simulates a dice game with multiple dice.
    
    Attributes:
        dice (list): A list of Die objects.
        play_data (pandas.DataFrame): Stores the results of the rolls.
    """
    
    def __init__(self, dice: list):
        """
        Initializes the game with a list of dice.
        
        Args:
            dice (list): A list of Die objects.
        
        Raises:
            TypeError: If any element in dice is not a Die object.
            ValueError: If the dice do not have the same faces.
        """
        if not all(isinstance(die, Die) for die in dice):
            raise TypeError("All elements must be Die objects")
        
        faces_set = {tuple(die.faces) for die in dice}
        if len(faces_set) != 1:
            raise ValueError("All dice must have the same faces.")
        
        self.dice = dice
        self.play_data = None
        
    def play(self, num_rolls):
        """
        Rolls all dice a specified number of times and stores the results.
        
        Args:
            num_rolls (int): The number of times to roll all dice.
        """
        results = {}
        for idx, die in enumerate(self.dice):
            results[idx] = die.roll(num_rolls)
        self.play_data = pd.DataFrame(results)
        return self.play_data
    
    def show_results(self, format='wide'):
        """
        Returns the results of the rolls in either wide or narrow format.
        
        Args:
            format (str): The format of the result, either 'wide' or 'narrow' (default is 'wide').
        
        Returns:
            pandas.DataFrame: A DataFrame with the results in the specified format.
        
        Raises:
            ValueError: If an invalid format is specified.
        """
        if self.play_data is None:
            raise ValueError("No results to show. Please play the game first.")
        if format == 'wide':
            return self.play_data
        elif format == 'narrow':
            return self.play_data.stack().reset_index(name='Outcome').rename(columns={'level_0': 'Roll', 'level_1': 'Die'})
        else:
            raise ValueError("Invalid format specified")


class Analyzer:
    """
    A class to analyze the results of a dice game.
    
    Attributes:
        game (Game): The Game object that stores the results to analyze.
        results (pandas.DataFrame): Stores the results of the game rolls.
    """
    
    def __init__(self, game: Game):
        """
        Initializes the analyzer with a game.
        
        Args:
            game (Game): A Game object.
        
        Raises:
            ValueError: If the input is not a Game object.
        """
        if not isinstance(game, Game):
            raise ValueError("The input must be a Game object.")
        self.game = game
        self.results = game.play_data
    
    def jackpot(self):
        """
        Computes the number of jackpots (all faces the same).
        
        Returns:
            int: The number of jackpots.
        """
        return sum(self.results.nunique(axis=1) == 1)
    
    def face_counts_per_roll(self):
        """
        Computes how many times each face is rolled in each event.
        
        Returns:
            pandas.DataFrame: A DataFrame with face counts for each roll in wide format.
        """
        return self.results.apply(lambda row: row.value_counts(), axis=1).fillna(0)
    
    def combo_count(self):
        """
        Computes distinct combinations of faces rolled and their counts.
        
        Returns:
            pandas.DataFrame: A DataFrame with combinations and counts, MultiIndex format.
        """
        return self.results.apply(lambda row: pd.Series([tuple(sorted(row))]), axis=1).value_counts()
    
    def permutation_count(self):
        """
        Computes distinct permutations of faces rolled and their counts.
        
        Returns:
            pandas.DataFrame: A DataFrame with permutations and counts, MultiIndex format.
        """
        return self.results.apply(lambda row: pd.Series([tuple(row)]), axis=1).value_counts()



In [11]:
# Data tests to ensure accuracy
fair_die = Die(np.array([1, 2, 3, 4, 5, 6]))

game = Game([fair_die] * 5)
game.play(1000)
print(game.show_results(format='wide').head())

analyzer = Analyzer(game)

print(f"Number of jackpots: {analyzer.jackpot()}")
print(analyzer.face_counts_per_roll().head())
print(analyzer.combo_count().head())
print(analyzer.permutation_count().head())

   0  1  2  3  4
0  3  1  4  1  1
1  2  3  3  3  4
2  1  5  5  1  5
3  4  4  6  4  4
4  6  6  3  2  3
Number of jackpots: 0
     1    2    3    4    5    6
0  3.0  0.0  1.0  1.0  0.0  0.0
1  0.0  1.0  3.0  1.0  0.0  0.0
2  2.0  0.0  0.0  0.0  3.0  0.0
3  0.0  0.0  0.0  4.0  0.0  1.0
4  0.0  1.0  2.0  0.0  0.0  2.0
(1, 2, 3, 4, 5)    23
(1, 2, 3, 4, 6)    16
(1, 2, 4, 5, 6)    15
(1, 1, 3, 4, 6)    15
(1, 2, 2, 4, 6)    14
Name: count, dtype: int64
(2, 6, 1, 2, 1)    3
(6, 4, 3, 1, 1)    3
(1, 5, 3, 4, 1)    3
(2, 1, 1, 2, 1)    3
(2, 4, 6, 6, 5)    2
Name: count, dtype: int64


In [7]:
class TestDie(unittest.TestCase):
    
    def test_initialize_die(self):
        die = Die(np.array([1, 2, 3, 4, 5, 6]))
        self.assertEqual(die.show().shape[0], 6)
        self.assertTrue(np.all(die.show()['weights'] == 1.0))
    
    def test_change_weight(self):
        die = Die(np.array([1, 2, 3, 4, 5, 6]))
        die.change_weight(3, 2.0)
        self.assertEqual(die.show().loc[3, 'weights'], 2.0)
    
    def test_invalid_change_weight(self):
        die = Die(np.array([1, 2, 3, 4, 5, 6]))
        with self.assertRaises(IndexError):
            die.change_weight(7, 2.0)
        with self.assertRaises(TypeError):
            die.change_weight(3, "invalid")
    
    def test_roll(self):
        die = Die(np.array([1, 2, 3, 4, 5, 6]))
        results = die.roll(5)
        self.assertEqual(len(results), 5)
        self.assertTrue(all(result in die.show().index for result in results))
    
    def test_show_state(self):
        die = Die(np.array([1, 2, 3, 4, 5, 6]))
        state = die.show()
        self.assertEqual(state.shape[0], 6)
        self.assertTrue(np.all(state['weights'] == 1.0))


class TestGame(unittest.TestCase):
    
    def test_initialize_game(self):
        die1 = Die(np.array([1, 2, 3, 4, 5, 6]))
        game = Game([die1])
        self.assertEqual(len(game.dice), 1)
    
    def test_play_game(self):
        die1 = Die(np.array([1, 2, 3, 4, 5, 6]))
        game = Game([die1])
        result = game.play(5)
        self.assertEqual(result.shape[0], 5)
        self.assertEqual(result.shape[1], 1)
    
    def test_show_results_wide(self):
        die1 = Die(np.array([1, 2, 3, 4, 5, 6]))
        game = Game([die1])
        game.play(5)
        result = game.show_results(format='wide')
        self.assertEqual(result.shape[0], 5)
        self.assertEqual(result.shape[1], 1)
    
    def test_show_results_narrow(self):
        die1 = Die(np.array([1, 2, 3, 4, 5, 6]))
        game = Game([die1])
        game.play(5)
        result = game.show_results(format='narrow')
        self.assertEqual(result.shape[0], 5)
        self.assertEqual(result.shape[1], 3)


class TestAnalyzer(unittest.TestCase):
    
    def test_analyze_jackpot(self):
        die1 = Die(np.array([1, 2, 3, 4, 5, 6]))
        game = Game([die1] * 5)
        game.play(1000)
        analyzer = Analyzer(game)
        jackpot_count = analyzer.jackpot()
        jackpot_rate = jackpot_count / 1000
        self.assertGreaterEqual(jackpot_rate, 0)
        self.assertLess(jackpot_rate, 0.05)
    
    def test_analyze_face_counts(self):
        die1 = Die(np.array([1, 2, 3, 4, 5, 6]))
        game = Game([die1])
        game.play(5)
        analyzer = Analyzer(game)
        face_counts = analyzer.face_counts_per_roll()
        self.assertEqual(face_counts.shape[0], 5)
    
    def test_analyze_combos(self):
        die1 = Die(np.array([1, 2, 3, 4, 5, 6]))
        game = Game([die1])
        game.play(5)
        analyzer = Analyzer(game)
        combos = analyzer.combo_count()
        self.assertTrue(combos.shape[0] > 0)

# Running the tests
if __name__ == '__main__':
    unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestDie))
    unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestGame))
    unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestAnalyzer))

.....
----------------------------------------------------------------------
Ran 5 tests in 0.005s

OK
....
----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK
...
----------------------------------------------------------------------
Ran 3 tests in 0.043s

OK
